# Stable Diffusion notebook by [@alexpopo](https://github.com/alexpopo/alex_stable_diffusion_webui)

This let's you generate images with CompVis/Stability [Stable Diffusion](https://github.com/CompVis/stable-diffusion) with bonus [KLMS sampling](https://github.com/crowsonkb/k-diffusion.git) from [@RiversHaveWings](https://twitter.com/RiversHaveWings)

You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/runwayml/stable-diffusion-v1-5).

Other Stable Diffusion tools:
* [Deforum Stable Diffusion](https://colab.research.google.com/github/deforum/stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb) by [deforum_art](https://twitter.com/deforum_art) team
* [HuggingFace space for Stable Diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
* [Stable Diffusion Interpolation](https://colab.research.google.com/drive/1EHZtFjQoRr-bns1It5mTcOVyZzZD9bBc?usp=sharing) by [@ygantigravity](https://twitter.com/ygantigravity)
* [Stable Diffusion with diffusers](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=zHkHsdtnry57)
* [Prompt-space Interpolation](https://colab.research.google.com/drive/1gj8_EPyntZuaiQuzYOzog5YKA74esoB3#scrollTo=3g5rSm43STsW) by [@DigThatData](https://twitter.com/DigThatData)
* [Stable Diffusion Lite](https://colab.research.google.com/drive/1cl3d84B5AXepycKAfCba77faUFBRGY2O?usp=sharing#scrollTo=Rn06QzsdsICR) by [@future__art](https://twitter.com/future__art)
* [Stable Diffusion web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui)
* More to come!!

If you're looking for more Ai art tools check here [Ai generative art tools list](https://pharmapsychotic.com/tools.html).

if you encounter any issues, feel free to discuss them.[Discord Support](https://discord.com/channels/1002292111942635562)

In [ ]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f4c39074-a716-dbbb-688a-0d48815dc7f1)


In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#@title Check Environment
#@markdown ##### 谷歌云端硬盘：仅第一次需要下载资料，之后持久保存在谷歌云端硬盘
#@markdown ##### 简单便捷方式：每次重新连接CoLab需要重新下载所有资料，不需要云硬盘

import os
import time

if os.path.exists("/content/gdrive/MyDrive/"):
  print('Gdrive connected, using Gdrive storage ...')
  print("仅第一次需要下载资料，之后持久保存在谷歌云端硬盘")
else:
  print('[1;31mGdrive not connected, using colab storage ...')
  print("每次重新连接CoLab需要重新下载所有资料，云硬盘不是必须的")

time.sleep(1)

outputs_path = "/content/SDWebUI"
%mkdir -p $outputs_path
print()
%cd $outputs_path

Gdrive connected, using Gdrive storage ...
仅第一次需要下载资料，之后持久保存在谷歌云端硬盘

/content/SDWebUI


In [ ]:
#@title First Base Requirements

# 在尝试安装之前，请确保满足所有必需的依赖项
print("Check Common Software Versions ...")
!git --version
print()
print("git-v2.12及之后版本集成了git-lfs")
print("-"*80)
!apt --version
print("-"*80)
!python3 --version
print("-"*80)
!wget --version | grep "^GNU Wget"
print("-"*80)

print('Installing base requirements...')

# Debian-based:
!sudo apt install python3-venv

Check Common Software Versions ...
git version 2.25.1

git-v2.12及之后版本集成了git-lfs
--------------------------------------------------------------------------------
apt 2.0.9 (amd64)
--------------------------------------------------------------------------------
Python 3.8.10
--------------------------------------------------------------------------------
GNU Wget 1.20.3 built on linux-gnu.
--------------------------------------------------------------------------------
Installing base requirements...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  python-pip-whl python3.8-venv
The following NEW packages will be installed:
  python-pip-whl python3-venv python3.8-venv
0 upgraded, 3 newly installed, 0 to remove and 28 not upgraded.
Need to get 1,812 kB

In [ ]:
#@title Next Clone WebUI Repo Code And Download SD Model

# clone web ui and go into its directory
!git clone --depth 1 --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui.git $outputs_path/stable-diffusion-webui


# test_outputs_path = "/content/TestSDWebUI"
# %mkdir -p $outputs_path
# !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git $test_outputs_path/stable-diffusion-webui

# Download Stable Diffusion Model Checkpoint
# 下载模型文件"sd-v1-4.ckpt"并进行hash校验，检查是否下载完整
models_path = "/content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion"

!wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O /content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion/model.ckpt

"""
Git LFS Details
SHA256: fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556
Pointer size: 135 Bytes
Size of remote file: 4.27 GB
"""

!sha256sum $models_path/model.ckpt

Cloning into '/content/SDWebUI/stable-diffusion-webui'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (210/210), done.
remote: Total 229 (delta 13), reused 84 (delta 8), pack-reused 0
Receiving objects: 100% (229/229), 1.39 MiB | 18.44 MiB/s, done.
Resolving deltas: 100% (13/13), done.
--2023-02-08 01:34:27--  https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e850:e203:c458:10cd:fc3c, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/4c/37/4c372b4ebb57bbd02e68413d4951aa326d4b3cfb6e62db989e529c6d4b26fb21/fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sd-v1-4.ckpt%3B+filena

In [ ]:
#@title Automatic Installation

%cd $outputs_path/stable-diffusion-webui
!bash webui.sh

/content/SDWebUI/stable-diffusion-webui

################################################################
Install script for stable-diffusion + Web UI
Tested on Debian 11 (Bullseye)
################################################################

################################################################
Running on root user
################################################################

################################################################
Repo already cloned, using it as install directory
################################################################

################################################################
Create and activate python venv
################################################################

################################################################
Launching launch.py...
################################################################
Python 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
Commit hash: ea9bd9fc7409109adcd61b897abc2

In [ ]:
# !pip3 list | grep "pip"
!pip3 list | grep -e "xformers" -e "scipy"
print("-"*80)

!source /content/SDWebUI/stable-diffusion-webui/venv/bin/activate

# Launching Web UI with arguments: No module 'xformers' Issues.
# xformers-0.0.16-cp38-cp38-manylinux2014_x86_64.whl
!pip install xformers==0.0.16rc425

scipy                         1.7.3
xformers                      0.0.16
--------------------------------------------------------------------------------
/content/SDWebUI/stable-diffusion-webui
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title
# Start Stable-Diffusion

%cd $outputs_path/stable-diffusion-webui
# !python3 webui.py --api --disable-safe-unpickle --enable-insecure-extension-access --no-half-vae  --ckpt "$model" --xformers --disable-console-progressbars
# !python3 webui.py --share --xformers
!python3 webui.py

/content/SDWebUI/stable-diffusion-webui
Traceback (most recent call last):
  File "webui.py", line 7, in <module>
    from fastapi import FastAPI
ModuleNotFoundError: No module named 'fastapi'
